In [1]:
import pandas as pd

from def_clean import DefensiveCleaning

def_cleaning = DefensiveCleaning(n_cuts=11)

..............................initializing
..data downloaded...


In [2]:
full_df, cut_df = def_cleaning.reduce_time(1)
cos_df = def_cleaning.cosine_closest_player(cut_df)

...Week 1 loaded...
...filtered...
...transformed...
...time reduced...
...closest player based on cosine similarity calculated...


In [4]:
def_cleaning.generate_full_df(1, 3)

...Week 1 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 1 COMPLETE.....

   33.33% COMPLETE   

--- 0.4 minutes elapsed ---

the weeks complete:  [1]
...Week 2 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 2 COMPLETE.....

   66.67% COMPLETE   

--- 0.9 minutes elapsed ---

the weeks complete:  [1 2]
...Week 3 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 3 COMPLETE.....

   100.0% COMPLETE   

--- 1.4 minutes elapsed ---

the weeks complete:  [1 2 3]
Defensive cleaning complete --- check assets/def_clean_output.csv


posId                                                                                        CBL0_act  \
gameId     playId defendersInTheBox numberOfPassRushers DB LB DL yardline_first yardline_100            
2018090600 75     7.0               4.0                 5  2  4  65.0           80.0                M   
           146    7.0               4.0                 5  2  4  29.0           39.0                M   
           168    6.0               4.0                 5  2  4  29.0           39.0                Z   
           190    6.0               5.0                 6  1  4  29.0           39.0                Z   
           256    8.0               6.0                 2  3  6  0.0            1.0               NaN   
...                                                                                               ...   
2018092400 4029   6.0               4.0                 5  2  4  73.0           64.0                M   
           4121   6.0               4.0                 5  4  2  70.0           80.0                Z   
           4165   5.0               5.0                 6  3  2  70.0           80.0                M   
           4199   6.0               4.0                 6  3  2  70.0           80.0                M   
           4286   7.0               5.0                 4  3  4  42.0           32.0                M   

posId                                                                                        CBL0_x_start  \
gameId     playId defendersInTheBox numberOfPassRushers DB LB DL yardline_first yardline_100                
2018090600 75     7.0               4.0                 5  2  4  65.0           80.0                 7.37   
           146    7.0               4.0                 5  2  4  29.0           39.0                 2.72   
           168    6.0               4.0                 5  2  4  29.0           39.0                 1.62   
           190    6.0               5.0                 6  1  4  29.0           39.0                 1.62   
           256    8.0               6.0                 2  3  6  0.0            1.0                   NaN   
...                                                                                                   ...   
2018092400 4029   6.0               4.0                 5  2  4  73.0           64.0                 0.98   
           4121   6.0               4.0                 5  4  2  70.0           80.0                 0.76   
           4165   5.0               5.0                 6  3  2  70.0           80.0                 4.25   
           4199   6.0               4.0                 6  3  2  70.0           80.0                 0.38   
           4286   7.0               5.0                 4  3  4  42.0           32.0                 0.88   

posId                                                                                        CBL0_y_start  \
gameId     playId defendersInTheBox numberOfPassRushers DB LB DL yardline_first yardline_100                
2018090600 75     7.0               4.0                 5  2  4  65.0           80.0                 6.13   
           146    7.0               4.0                 5  2  4  29.0           39.0                 0.59   
           168    6.0               4.0                 5  2  4  29.0           39.0                 5.45   
           190    6.0               5.0                 6  1  4  29.0           39.0                 4.85   
           256    8.0               6.0                 2  3  6  0.0            1.0                   NaN   
...                                                                                                   ...   
2018092400 4029   6.0               4.0                 5  2  4  73.0           64.0                19.99   
           4121   6.0               4.0                 5  4  2  70.0           80.0                 2.06   
           4165   5.0               5.0                 6  3  2  70.0           80.0                 4.94   
           4199  

In [26]:
import numpy as np

cut_df['distance'] = np.linalg.norm(cut_df[['y_dir_qb_def', 'x_behind_line_def']].values -
                                    cut_df[['y_dir_qb_off', 'x_behind_line_off']].values, axis=1)

In [34]:
df_rec = full_df[['gameId', 'playId', 'posId', 'receiver']].drop_duplicates().rename(
    {'posId': 'posId_off'}, axis=1)
cut_rec = cut_df.merge(df_rec, on=['gameId', 'playId', 'posId_off'])
cut_rec_in = cut_rec[cut_rec['receiver']]

In [35]:
cut_rec_in

,gameId,playId,posId_def,time_cut,y_dir_qb_def,x_behind_line_def,off_def,posId_off,y_dir_qb_off,x_behind_line_off,off_off,distance,receiver
154,2018090600,75,CBL0,1,6.130000,7.376667,False,TEL0,4.776667,-1.296667,True,8.778281,True
155,2018090600,75,CBR0,1,-16.806667,6.086667,False,TEL0,4.776667,-1.296667,True,22.811267,True
156,2018090600,75,FSL0,1,4.603333,3.510000,False,TEL0,4.776667,-1.296667,True,4.809791,True
157,2018090600,75,FSR0,1,-2.073333,16.396667,False,TEL0,4.776667,-1.296667,True,18.973048,True
158,2018090600,75,LBR0,1,-0.130000,4.296667,False,TEL0,4.776667,-1.296667,True,7.440481,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
374278,2018091001,4192,FSL0,11,4.300000,18.410000,False,WRL2,17.750000,2.835000,True,20.578706,True
374279,2018091001,4192,ILBR0,11,-14.195000,5.110000,False,WRL2,17.750000,2.835000,True,32.025906,True
374280,2018091001,4192,OLBL0,11,-1.840000,-2.150000,False,WRL2,17.750000,2.835000,True,20.214310,True
374281,2018091001,4192,SSL0,11,-1.605000,5.760000,False,WRL2,17.750000,2.835000,True,19.574771,True


In [25]:
dist_df

,time,x,y,s,a,dis,o,dir,event,nflId,...,exceeded_1yd,receiver,y_dir,y_dir_qb,y_dir_qb_starting,time_cut,pos_off_closest,min_dist_rec,dist_qb,rec_closer
0,2018-09-07 01:07:15.599000+00:00,88.89,36.50,0.01,0.01,0.00,104.49,36.45,ball_snap,79848.0,...,True,False,16.80,-9.87,-9.87,1,WRR0,1.920339,NaN,False
1,2018-09-07 01:07:15.599000+00:00,91.36,44.14,0.00,0.01,0.02,289.53,49.86,ball_snap,2495454.0,...,True,True,9.16,-17.51,-17.51,1,NaN,NaN,NaN,False
2,2018-09-07 01:07:15.599000+00:00,86.47,22.03,0.18,0.07,0.01,71.89,82.30,ball_snap,2495613.0,...,True,False,31.27,4.60,4.60,1,TEL0,4.833353,NaN,False
3,2018-09-07 01:07:15.599000+00:00,90.78,36.16,0.00,0.00,0.00,259.19,294.93,ball_snap,2533040.0,...,True,True,17.14,-9.53,-9.53,1,NaN,NaN,NaN,False
4,2018-09-07 01:07:15.599000+00:00,73.61,28.70,0.02,0.02,0.00,110.09,258.40,ball_snap,2534832.0,...,True,False,24.60,-2.07,-2.07,1,WRR0,18.720590,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341044,2018-09-11 05:29:36+00:00,26.66,7.72,2.64,0.83,0.26,49.10,282.71,pass_forward,2552488.0,...,True,False,45.58,16.16,16.26,11,WRL2,11.308094,NaN,False
341045,2018-09-11 05:29:36+00:00,35.19,25.58,2.14,1.38,0.22,91.21,333.38,pass_forward,2556277.0,...,True,False,27.72,-1.70,-0.50,11,WRL0,4.117524,NaN,False
341046,2018-09-11 05:29:36+00:00,35.85,38.20,2.68,1.33,0.26,79.60,343.60,pass_forward,2556593.0,...,True,False,15.10,-14.32,-12.53,11,TER0,2.834784,NaN,False
341047,2018-09-11 05:29:36+00:00,40.85,46.18,0.92,4.07,0.08,156.73,134.27,pass_forward,2556885.0,...,False,False,7.12,-22.30,-22.42,11,NaN,NaN,NaN,False


In [12]:
import numpy as np

cut_df['distance'] = np.linalg.norm(cut_df[['y_dir_qb_def', 'x_behind_line_def']].values -
                                    cut_df[['y_dir_qb_off', 'x_behind_line_off']].values, axis=1)

In [22]:
closest_players = cut_df.sort_values('distance').groupby(
    ['gameId', 'playId', 'posId_def', 'time_cut']).head(3)

In [24]:
def_cleaning.cosine_closest_player(closest_players)

...closest player based on cosine similarity calculated...


,gameId,playId,posId_def,time_cut_time2,posId_off_max,posId_off_min
0,2018090600,75,CBL0,2,TEL0,RBL0
1,2018090600,75,CBL0,3,TEL0,RBL0
2,2018090600,75,CBL0,4,TEL0,FBR0
3,2018090600,75,CBL0,5,TEL0,FBR0
4,2018090600,75,CBL0,6,TEL0,FBR0
...,...,...,...,...,...,...
61885,2018091001,4192,SSR0,7,TER0,RBR0
61886,2018091001,4192,SSR0,8,TER0,RBR0
61887,2018091001,4192,SSR0,9,TER0,RBR0
61888,2018091001,4192,SSR0,10,WRL0,RBR0


In [2]:
pos_df = def_cleaning.weeks_data.copy()

In [5]:
import numpy as np

game_id = pos_df['gameId'].iloc[1902]
play_id = pos_df['playId'].iloc[1902]
n_cuts = 11

In [9]:
from play_vis import create_full_chart, create_starting_chart

create_starting_chart(pos_df, game_id, play_id)


Play description: (Shotgun) M.Ryan pass incomplete short left to D.Freeman.


alt.LayerChart(...)

In [147]:
import altair as alt

def create_full_chart(pos_df, game_id, play_id, n_cuts=11):
    play_df = pos_df[(pos_df['gameId'] == game_id) & (pos_df['playId'] == play_id)]
    plays_df = pd.read_csv('Kaggle-Data-Files/plays.csv')
    after_snap = play_df[play_df['time'].ge(play_df[play_df['event'] == 'ball_snap']['time'].iloc[0])]
    after_snap_play = after_snap.merge(plays_df, on=['gameId', 'playId'])
    after_snap_play['time_diff'] = after_snap_play.groupby('displayName')['time'].diff()
    after_snap_play['time_diff'][after_snap_play['time_diff'].isnull()] = pd.Timedelta(0)
    after_snap_play['time_acc_s'] = after_snap_play['time_diff'].dt.microseconds.div(1e6)
    after_snap_play['time_acc_s'] = after_snap_play.groupby('displayName')['time_acc_s'].transform(lambda x: x.cumsum())
    n_times = after_snap_play['time_acc_s'].unique().shape[0]
    if n_times < n_cuts:
        after_snap_play['time_cuts'] = pd.cut(after_snap_play['time_acc_s'], bins=n_times,
                                              labels=range(1, n_times+1))
    else:
        after_snap_play['time_cuts'] = pd.cut(after_snap_play['time_acc_s'], bins=n_cuts,
                                              labels=range(1, n_cuts+1))
    after_snap_play['x_100'] = after_snap_play['x'].sub(10)
    off_def_map = after_snap_play.groupby('team').apply(
        lambda x: 'QB' in x['position'].values).reset_index().rename({0: 'off'}, axis=1)

    off_def_map.replace({True: 'Offense', False: 'Defense'}, inplace=True)
    after_snap_off = after_snap_play.merge(off_def_map, on='team')
    after_snap_off['yardline_100'] = after_snap_off['absoluteYardlineNumber'].sub(10)

    # Find which side the offense is lined up on.
    try:
        on_left = after_snap_off[after_snap_off['position'] == 'QB'][
                      'x_100'].iloc[0] < after_snap_off['yardline_100'].iloc[0]
    except IndexError:
        try:
            on_left = (after_snap_off[after_snap_off['position'] == 'P']['x_100'].iloc[0]
                       < after_snap_off['yardline_100'].iloc[0])
            team_map = after_snap_off.groupby(['gameId', 'playId', 'team']).apply(lambda x: 'P' in x['position'].unique())
            team_map = team_map.map(lambda x: 'Offense' if x else 'Defense')
            team_map.name = 'off'
            after_snap_off.drop('off', axis=1, inplace=True)
            after_snap_off = after_snap_off.merge(
                team_map, left_on=['gameId', 'playId', 'team'], right_index=True)
        except IndexError:
            on_left = after_snap_off['x_100'].iloc[0] > after_snap_off['yardline_100'].iloc[0]


    if on_left == False:
        side = 'right'
    else:
        side = 'left'

    if side == 'left':
        after_snap_off['yardline_first'] = after_snap_off['yardline_100'].add(after_snap_off['yardsToGo'])
    else:
        after_snap_off['yardline_first'] = after_snap_off['yardline_100'].sub(after_snap_off['yardsToGo'])
    if side == 'right':
        after_snap_off['x_dir'] = after_snap_off['x_100'].rsub(100)
        after_snap_off['yardline_dir'] = after_snap_off['yardline_100'].rsub(100)
        after_snap_off['yardline_first_dir'] = after_snap_off['yardline_first'].rsub(100)
        after_snap_off['y_dir'] = after_snap_off['y'].copy()
    else:
        after_snap_off['x_dir'] = after_snap_off['x_100'].copy()
        after_snap_off['yardline_dir'] = after_snap_off['yardline_100'].copy()
        after_snap_off['yardline_first_dir'] = after_snap_off['yardline_first'].copy()
        after_snap_off['y_dir'] = after_snap_off['y'].rsub(53.3)
    after_snap_off['position'] = np.where(after_snap_off['displayName'] == 'Football',
                                          'Ft',
                                          after_snap_off['position'])
    after_snap_off['off'] = np.where(after_snap_off['position'] == 'Ft', 'Football',
                                     after_snap_off['off'])
    mean_pos = after_snap_off.groupby(['gameId', 'playId', 'time_cuts', 'displayName'])[
        ['x_dir', 'y_dir']].mean()
    per_timecut = mean_pos.merge(after_snap_off[['gameId', 'playId', 'time_cuts', 'displayName', 'position', 'off', 'yardline_dir', 'yardline_first_dir']].
                                 drop_duplicates(),
                                 left_index=True, right_on=['gameId', 'playId', 'time_cuts', 'displayName'])
    per_timecut['time_cuts'] = per_timecut['time_cuts'].astype(int)
    input_dropdown = alt.binding_select(options=per_timecut['time_cuts'].unique(), name='Time cut\n')
    selection = alt.selection_single(fields=['time_cuts'], bind=input_dropdown, init={'time_cuts': 1})

    points = alt.Chart(per_timecut).encode(x=alt.X('y_dir:Q', scale=alt.Scale(domain=[0, 53.3]), title=None),
                                  y=alt.Y('x_dir:Q', scale=alt.Scale(zero=False), title='Yardline'),
                                  color=alt.Color('off:N'),
                                  tooltip=['position:N']
                                  ).mark_circle(size=100, dx=5, dy=5)

    text = alt.Chart(per_timecut).encode(x=alt.X('y_dir:Q', scale=alt.Scale(domain=[0, 53.3]), title=None),
                                         y=alt.Y('x_dir:Q', scale=alt.Scale(zero=False), title='Yardline'),
                                         text=alt.Text('position:N')
                                         ).mark_text(dx=-15, fontSize=8)
    yardline_100 = alt.Chart(per_timecut).encode(y=alt.Y('yardline_dir:Q')).mark_rule(color='blue')
    yardline_first = alt.Chart(per_timecut).encode(y=alt.Y('yardline_first_dir:Q')).mark_rule(color='#ECEC01')
    chart = (points + text + yardline_first + yardline_100).add_selection(
        selection
    ).transform_filter(
        selection).properties(width=53.3*10,
                              height=(per_timecut['x_dir'].max() - per_timecut['x_dir'].min())*10)
    return chart

In [148]:
create_full_chart(pos_df, game_id, play_id)

alt.LayerChart(...)

In [153]:
play_df = pd.read_csv('Kaggle-Data-Files/plays.csv')

In [154]:
play_df

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,2018122200,2300,(7:53) J.Johnson pass incomplete short left [K...,3,2,5,WAS,play_type_unknown,WAS,31,...,NaN,NaN,NaN,DH,TEN 24,I,0,5,0.848703,False
19235,2018122200,3177,(6:53) (Shotgun) B.Gabbert pass incomplete sho...,4,3,7,TEN,play_type_unknown,WAS,37,...,NaN,NaN,NaN,DH,WAS 31,I,0,5,1.227695,False
19236,2018122201,566,(5:32) (Shotgun) P.Rivers pass deep right to K...,1,3,4,LAC,play_type_unknown,LAC,49,...,NaN,NaN,NaN,OPI,LAC 13,C,27,-10,-0.720168,False
19237,2018122201,1719,(1:08) P.Rivers pass incomplete deep middle to...,2,3,1,LAC,play_type_unknown,LAC,48,...,NaN,NaN,NaN,DPI,BAL 23,I,0,15,1.336078,True
